In [ ]:
import pandas as pd

In [ ]:
df_labels = pd.read_json('../data/output.manifest', lines=True).set_index('id')
df = (df_labels
      .assign(label=lambda df: df['rent-classification'] == 0)
      .rename(columns={'source': 'text'})
      [['label', 'text']]
     )

In [ ]:
df.astype({'label': 'int'}).to_csv('../data/train.csv', index=False)

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict

dataset = DatasetDict({
    'train': Dataset.from_pandas(df.head(700)),
    'validation': Dataset.from_pandas(df.tail(300)),
})
dataset

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
from transformers import TextClassificationPipeline

In [ ]:
clf = TextClassificationPipeline(model=model, tokenizer=tokenizer)

In [ ]:
%%time
preds = clf(df.tail(300)['text'].tolist())

In [ ]:
pd.DataFrame(preds)

In [ ]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
import plotly.express as px

In [ ]:
y_val = df.tail(300)['label'].values

In [ ]:
y_pred = (pd.DataFrame(preds)['label'] == 'LABEL_1').values

In [ ]:
precision_recall_fscore_support(
    y_val,
    y_pred
)

In [ ]:
def plot_cm(y_true, y_pred):
    labels = sorted(pd.unique(y_true))
    df = pd.DataFrame(confusion_matrix(y_true, y_pred), index=labels, columns=labels)
    return px.imshow(df, labels={'y': 'actual', 'x': 'pred'}, text_auto=True)

In [ ]:
import plotly.io as pio
pio.renderers.default='notebook'

In [ ]:
plot_cm(y_val, y_pred)

In [ ]:
df.tail(300).assign(pred=y_pred).sample(20).style

In [ ]:
df.tail(300).assign(pred=y_pred)[y_val != y_pred].style

In [ ]:
model.save_pretrained('../model')

tokenizer.save_pretrained('../tokenizer')

In [ ]:
model